In [232]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [233]:
class SphereDatasetAnchor(Dataset):
    def __init__(self, extra_modes, num_points, transform_fn=None):
        self.n = num_points
        self.m = extra_modes + 1
        self.points = np.random.uniform(-1,1,size=(self.n, 3))
        mags = np.linalg.norm(self.points, axis=-1)
        self.points /= mags[...,None]
        self.disks = np.array([np.random.uniform(-1,1,size=(3)) for i in range(extra_modes)])
        self.disks = np.array([disk / np.linalg.norm(disk) for disk in self.disks])
        self.disk_points = torch.tensor([self.project_points(self.points, disk) for disk in self.disks]).float()
        self.points = torch.tensor(self.points).float()
        self.disks = torch.tensor(self.disks).float()
        self.transform_fn = transform_fn
        
    def project_points(self, points, disk):
        output_points = []
        for point in points:
            cosval = np.dot(point, disk)
            arrow = (point*1/np.dot(point, disk) - disk) * cosval # mind the sign of cosval
            arrow /= np.linalg.norm(arrow)
            output_points.append(arrow)
        return np.array(output_points)
    
    def __len__(self):
        return self.m-1
    
    def __getitem__(self, idx):
        e_points = self.disk_points[idx]
        if self.transform_fn is not None:
            return self.transform_fn.evaluate(self.points), self.transform_fn.evaluate(e_points)
        return self.points, e_points
    
    def all_points(self):
        return [self.points] + list(self.disk_points)

In [234]:
class Mapper(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 128)
    
    def forward(self, X):
        return self.linear(X)

    def evaluate(self, X):
        self.eval()
        with torch.no_grad():
            return self.forward(X)

mapper = Mapper()

In [235]:
dataset = SphereDatasetAnchor(3, 1000, transform_fn=mapper)

In [261]:
from torch.nn.functional import cross_entropy
class ContrastiveModel(nn.Module):
    def __init__(self, modes):
        super().__init__()
        self.linears = nn.ModuleList([nn.Linear(128, 3) for i in range(modes)])
        self.comps = nn.ModuleList([nn.Linear(3*2, 1) for i in range(modes-1)])
    
    def loss(self, corr_mat):
        losses_row = cross_entropy(corr_mat, torch.arange(0, len(corr_mat)).long())
        losses_col = cross_entropy(torch.transpose(corr_mat,0,1), torch.arange(0, len(corr_mat)).long())
        return torch.mean(losses_row) + torch.mean(losses_col)
    
    def compute_corr_dot(self, a_points, b_points):
        return torch.sum(a_points[:,None] * b_points[None,:], dim=-1)
    
    def compute_corr_func(self, a_points, b_points, b):
        a_points_tiled = torch.tile(a_points[:,None], (1,len(b_points),1))
        b_points_tiled = torch.tile(b_points[None,:], (len(a_points),1,1))
        return self.comps[b-1](torch.concat([a_points_tiled, b_points_tiled], dim=-1))[:,:,0]
    
    def forward(self, a, b, a_points, b_points):
        a_points = self.linears[0](a_points)
        b_points = self.linears[0](b_points)
        corr_mat = self.compute_corr_dot(a_points, b_points)
        return corr_mat
    
    def run_evaluate(self, i, points):
        return self.linears[0](points)

In [262]:
model = ContrastiveModel(4)

In [263]:
from torch.optim import Adam
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
optim = Adam(model.parameters(), lr=0.01)

In [264]:
def train_data(optim, model, ptrs1, ptrs2, a, b, iteration=10):
    avg_loss = 0
    for i in range(iteration):
        optim.zero_grad()
        corr_mat = model(a, b, ptrs1, ptrs2)
        loss = model.loss(corr_mat)
        loss.backward()
        optim.step()
        avg_loss += loss.item() / iteration
    return avg_loss

In [265]:
EPOCH = 20
for epoch in range(EPOCH):
    for i in range(3):
        ptrs1, ptrs2 = dataset[i]
        loss = train_data(optim, model, ptrs1, ptrs2, 0, i+1, iteration=10)
        print(f'{i+1}: {loss}')

1: 11.89713020324707
2: 12.162207221984863
3: 11.205359554290771
1: 12.774119949340822
2: 11.316295433044434
3: 11.124613761901855
1: 11.189048385620115
2: 10.93609113693237
3: 10.930501651763915
1: 11.011807537078857
2: 10.906158065795898
3: 10.91740379333496
1: 11.01710023880005
2: 10.896265506744385
3: 10.914795684814454
1: 11.00171127319336
2: 10.89536781311035
3: 10.912680244445802
1: 11.003368186950684
2: 10.89579792022705
3: 10.91152696609497
1: 11.004448032379152
2: 10.89976234436035
3: 10.912063217163086
1: 11.004242038726808
2: 10.903711605072022
3: 10.913156509399414
1: 11.004485702514648
2: 10.907089614868164
3: 10.914958858489992
1: 11.006492233276369
2: 10.911806869506837
3: 10.918433094024659
1: 11.011660194396974
2: 10.919288730621338
3: 10.924265003204345
1: 11.021223735809325
2: 10.930354309082032
3: 10.932452487945557
1: 11.035862159729003
2: 10.944055271148683
3: 10.941085338592531
1: 11.05307445526123
2: 10.956311035156249
3: 10.9466046333313
1: 11.065345478057862


In [266]:
orig_points = dataset.all_points()

In [267]:
import plotly.express as px
import pandas as pd

In [268]:
df = pd.DataFrame(np.concatenate(orig_points, axis=0), columns=['x', 'y', 'z'])
df['type'] = ['0']*1000 + ['1']*1000 + ['2']*1000 + ['3']*1000
fig = px.scatter_3d(df, x='x', y='y', z='z', color='type')
fig.show()

In [269]:
pred_points = [model.run_evaluate(i, mapper(points)).detach().numpy() for i, points in enumerate(orig_points)]

In [270]:
df = pd.DataFrame(np.concatenate(pred_points, axis=0), columns=['x', 'y', 'z'])
df['type'] = ['0']*1000 + ['1']*1000 + ['2']*1000 + ['3']*1000
fig = px.scatter_3d(df, x='x', y='y', z='z', color='type')
fig.show()

In [271]:
ptrs1 = dataset.points
ptrs2 = dataset.disk_points[2]
loss_total = 0
for i,ptr2 in enumerate(ptrs2):
    loss_total += cross_entropy(torch.sum(ptrs1 * ptr2[None,:], dim=-1), torch.tensor(i).long())
for i,ptr1 in enumerate(ptrs1):
    loss_total += cross_entropy(torch.sum(ptr1[None,:] * ptrs2, dim=-1), torch.tensor(i).long())
print(loss_total/len(ptrs2))

tensor(12.5478)


In [272]:
l1 = cross_entropy(torch.sum(ptrs1[:,None]*ptrs2[None,:], dim=-1), torch.arange(len(ptrs1)).long())
l2 = cross_entropy(torch.sum(ptrs2[:,None]*ptrs1[None,:], dim=-1), torch.arange(len(ptrs1)).long())
l1+l2

tensor(12.5478)

In [273]:
print(dataset.disks[0])

tensor([-0.3848,  0.8510, -0.3575])
